Certainly! Let’s break down the differences between a traditional Monte Carlo Tree Search (MCTS) and one that uses Upper Confidence Bounds for Trees (UCT).

### Key Concepts in MCTS and UCT

#### 1. **Basic Monte Carlo Tree Search (MCTS)**

Traditional MCTS involves four main steps:

1. **Selection**: Traverse the tree from the root to a leaf node, choosing child nodes according to some policy. In the simplest case, this could be random selection or a basic heuristic.

2. **Expansion**: If the selected node is not terminal and has unexplored actions, expand it by adding one or more child nodes.

3. **Simulation**: Perform a random simulation (default policy) from the newly expanded node to the end of the game or episode to estimate the outcome. This step often involves rolling out or simulating the game to completion using random actions or a fixed policy.

4. **Backpropagation**: Update the node values along the path from the newly expanded node back to the root. Typically, this involves updating statistics like visit counts and accumulated rewards.

#### 2. **Upper Confidence Bounds for Trees (UCT)**

UCT is an enhancement of traditional MCTS that adds a more sophisticated policy for node selection. UCT introduces the Upper Confidence Bound (UCB) formula to balance exploration and exploitation:

1. **Selection**: Instead of purely random selection or basic heuristics, UCT uses the UCB formula to choose child nodes. The formula is:
   \[
   \text{UCT Value} = \frac{\text{Total Simulation Reward}}{\text{Number of Visits}} + C \cdot \sqrt{\frac{\log(\text{Parent Visits})}{\text{Number of Visits}}}
   \]
   Here, the first term represents exploitation (favoring nodes with high average rewards), and the second term represents exploration (favoring nodes that have been visited less often).

2. **Expansion**: Similar to traditional MCTS, but often managed more systematically due to UCT's guidance on which nodes to explore.

3. **Simulation**: Can be the same as in traditional MCTS, where you simulate the outcome from the new node. 

4. **Backpropagation**: Updates the node values similarly to traditional MCTS, using the rewards obtained from the simulation.

### Differences in Implementation

#### **Selection**

- **Traditional MCTS**: Node selection might be random or based on simple heuristics. For instance, you could select an unexplored action randomly or pick a child node at random.

- **UCT-based MCTS**: Uses the UCB formula to select nodes, providing a balance between exploring less-visited nodes and exploiting nodes with higher rewards. This helps in more systematically exploring the search space.

#### **Expansion**

- **Traditional MCTS**: Expansion is often performed by adding one child node at a time, which could be any available action not yet tried.

- **UCT-based MCTS**: Expansion is guided by the selection process. UCT often suggests which nodes are most promising to expand based on the balance between exploration and exploitation.

#### **Simulation**

- **Traditional MCTS**: The simulation or default policy is typically random or semi-random, where you might just perform random actions until the end of the episode.

- **UCT-based MCTS**: The simulation policy might be the same, but because the selection process is more informed, the simulations are potentially more targeted based on prior knowledge.

#### **Backpropagation**

- **Traditional MCTS**: Updates the statistics (number of visits and total rewards) of nodes along the path from the newly expanded node to the root.

- **UCT-based MCTS**: Backpropagation is the same, but the node statistics being updated are influenced by the UCT-guided selection process.

### Summary

- **Traditional MCTS** is simpler and uses a basic or random policy for node selection. It relies heavily on the expansion and simulation steps to explore the search space.
- **UCT-based MCTS** incorporates the UCB formula to provide a more balanced and informed approach to node selection, leading to more efficient exploration and potentially better performance in complex scenarios.

The provided implementation of the traditional MCTS version is a straightforward approach without the UCB-based selection strategy, focusing on basic tree expansion, random simulations, and updating node values through backpropagation.

In [2]:
import uuid
import random
import gym
import numpy as np
from math import sqrt, log

class Node:
    def __init__(self, state, action, action_space, reward, terminal):
        self.identifier = str(uuid.uuid1())
        self.parent_identifier = None
        self.children_identifiers = []
        self.untried_actions = list(range(action_space))
        self.state = state
        self.total_simulation_reward = 0
        self.num_visits = 0
        self.performance = 0
        self.action = action
        self.reward = reward
        self.terminal = terminal

    def __str__(self):
        return "{}: (action={}, visits={}, reward={:d}, ratio={:0.4f})".format(
                                                  self.state,
                                                  self.action,
                                                  self.num_visits,
                                                  int(self.total_simulation_reward),
                                                  self.performance)

    def untried_action(self):
        action = random.choice(self.untried_actions)
        self.untried_actions.remove(action)
        return action

def vertical_lines(last_node_flags):
    vertical_lines = []
    vertical_line = '\u2502'
    for last_node_flag in last_node_flags[0:-1]:
        if last_node_flag == False:
            vertical_lines.append(vertical_line + ' ' * 3)
        else:
            # space between vertical lines
            vertical_lines.append(' ' * 4)
    return ''.join(vertical_lines)

def horizontal_line(last_node_flags):
    horizontal_line = '\u251c\u2500\u2500 '
    horizontal_line_end = '\u2514\u2500\u2500 '
    if last_node_flags[-1]:
        return horizontal_line_end
    else:
        return horizontal_line

In [3]:
class Tree:
    def __init__(self):
        self.nodes = {}
        self.root = None

    def size(self):
        """Returns the number of nodes in the tree."""
        return len(self.nodes)

    def depth_info(self):
        """Calculates the maximum depth and average depth of the tree."""
        depths = []

        def calculate_depth(node, current_depth):
            depths.append(current_depth)
            for child_id in node.children_identifiers:
                calculate_depth(self.nodes[child_id], current_depth + 1)

        if self.root is not None:
            calculate_depth(self.root, 0)

        if depths:
            max_depth = max(depths)
            avg_depth = sum(depths) / len(depths)
            return max_depth, avg_depth
        else:
            return 0, 0  # In case the tree is empty

    def is_expandable(self, node):
        if node.terminal:
            return False
        if len(node.untried_actions) > 0:
            return True
        return False

    def iter(self, identifier, depth, last_node_flags):
        if identifier is None:
            node = self.root
        else:
            node = self.nodes[identifier]

        if depth == 0:
            yield "", node
        else:
            yield vertical_lines(last_node_flags) + horizontal_line(last_node_flags), node

        children = [self.nodes[identifier] for identifier in node.children_identifiers]
        last_index = len(children) - 1

        depth += 1
        for index, child in enumerate(children):
            last_node_flags.append(index == last_index)
            for edge, node in self.iter(child.identifier, depth, last_node_flags):
                yield edge, node
            last_node_flags.pop()

    def add_node(self, node, parent=None):
        self.nodes.update({node.identifier: node})

        if parent is None:
            self.root = node
            self.nodes[node.identifier].parent = None
        else:
            self.nodes[parent.identifier].children_identifiers.append(node.identifier)
            self.nodes[node.identifier].parent_identifier=parent.identifier

    def children(self, node):
        children = []
        for identifier in self.nodes[node.identifier].children_identifiers:
            children.append(self.nodes[identifier])
        return children

    def parent(self, node):
        parent_identifier = self.nodes[node.identifier].parent_identifier
        if parent_identifier is None:
            return None
        else:
            return self.nodes[parent_identifier]

    def show(self):
        lines = ""
        for edge, node in self.iter(identifier=None, depth=0, last_node_flags=[]):
            lines += "{}{}\n".format(edge, node)
        print(lines)

    def render_policy(self):
        node = self.tree.root
        path = []
        print("Rendering final policy...\n")

        while node and not node.terminal:
            print(node)
            path.append(node.state)
            node = max(self.tree.children(node), key=lambda n: n.num_visits)
        
        if node:
            print(node)
            path.append(node.state)
        
        print("\nFinal policy path (states):", path)


In [4]:
class MonteCarloTreeSearch:
    def __init__(self, env, tree):
        self.env = env
        self.tree = tree
        self.action_space = self.env.action_space.n
        state = self.env.reset()
        self.tree.add_node(Node(state=state, action=None, action_space=self.action_space, reward=0, terminal=False))

    def expand(self, node):
        action = node.untried_action()
        state, reward, done, _, _ = self.env.step(action)
        new_node = Node(state=state, action=action, action_space=self.action_space, reward=reward, terminal=done)
        self.tree.add_node(new_node, node)
        return new_node



    def default_policy(self, node):
        if node.terminal:
            return node.reward

        while True:
            action = random.randint(0, self.action_space-1)
            state, reward, done, _, _ = self.env.step(action)
            if done:
                return reward

    def tree_policy(self):
        node = self.tree.root
        while not node.terminal:
            if self.tree.is_expandable(node):
                return self.expand(node)
            else:
                # Select a random child instead of using UCT-based selection
                node = random.choice(self.tree.children(node))
                state, reward, done, _, _ = self.env.step(node.action)
                assert node.state == state
        return node



    def backward(self, node, value):
        while node:
            node.num_visits += 1
            node.total_simulation_reward += value
            node.performance = node.total_simulation_reward / node.num_visits
            node = self.tree.parent(node)

    def forward(self):
        self._forward(self.tree.root)

    def _forward(self, node):
        best_child = max(self.tree.children(node), key=lambda n: n.performance, default=None)
        
        print("****** {} ******".format(best_child.state))

        for child in self.tree.children(best_child):
            print("{}: {:0.4f}".format(child.state, child.performance))

        if best_child and len(self.tree.children(best_child)) > 0:
            self._forward(best_child)

    def render_policy(self):
        node = self.tree.root
        path = []
        directions = {0: 'Left', 1: 'Down', 2: 'Right', 3: 'Up'}
        print("Rendering final policy...\n")
        
        # Close the current environment if open and reset with rendering enabled
        self.env.close()
        self.env = gym.make('FrozenLake-v1', is_slippery=True, render_mode='human')
        self.env.reset()

        while node and not node.terminal:
            self.env.render()
            path.append(node.state)
            children = self.tree.children(node)
            
            if not children:
                print("No more actions available.")
                break
            
            best_child = max(children, key=lambda n: n.performance, default=None)
            if best_child is not None and best_child.action is not None:
                print(f"Action: {directions.get(best_child.action, 'Unknown')} -> State: {best_child.state}")
                self.env.step(best_child.action)  # Take the action in the environment
                node = best_child
            else:
                print("No best child found.")
                break

        # Final rendering after reaching the goal
        self.env.render()
        self.env.close()    
        
        path2 = [item[0] if isinstance(item, tuple) else item for item in path]
        print(f"\nFinal policy path (states): {path2}")
        
        return path2

Training e testing


In [10]:
import json

def main():
    env = gym.make('FrozenLake-v1', is_slippery=False)
    tree = Tree()
    monteCarloTreeSearch = MonteCarloTreeSearch(env=env, tree=tree)
    results = []
   
    
    for steps in [1000, 5000, 10000]:
        
        for _ in range(0, steps):
            env.reset()
            node = monteCarloTreeSearch.tree_policy()
            reward = monteCarloTreeSearch.default_policy(node)
            monteCarloTreeSearch.backward(node, reward)

        # Measure and display tree statistics
        tree_size = tree.size()
        max_depth, avg_depth = tree.depth_info()

        monteCarloTreeSearch.tree.show()
        print("Best child choices:")
        monteCarloTreeSearch.forward()
        
        policy_path = monteCarloTreeSearch.render_policy()

        print(f"Tree size: {tree_size}")
        print(f"Maximum depth: {max_depth}")
        print(f"Average depth: {avg_depth:.2f}")

        # Save the results for this combination
        results.append({
            # 'steps': steps,
            'tree_size': tree_size,
            'max_depth': max_depth,
            'avg_depth': avg_depth,
            'policy_path': policy_path
        })

    
    

    # Save results to a JSON file
    with open('mcts_results.json', 'w') as json_file:
        json.dump(results, json_file, indent=4)

if __name__ == "__main__":
    main()

(0, {'prob': 1}): (action=None, visits=1000, reward=8, ratio=0.0080)
├── 1: (action=2, visits=266, reward=3, ratio=0.0113)
│   ├── 0: (action=0, visits=56, reward=1, ratio=0.0179)
│   │   ├── 1: (action=2, visits=12, reward=0, ratio=0.0000)
│   │   │   ├── 5: (action=1, visits=2, reward=0, ratio=0.0000)
│   │   │   ├── 1: (action=3, visits=2, reward=0, ratio=0.0000)
│   │   │   │   └── 0: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   ├── 2: (action=2, visits=2, reward=0, ratio=0.0000)
│   │   │   │   └── 2: (action=3, visits=1, reward=0, ratio=0.0000)
│   │   │   └── 0: (action=0, visits=5, reward=0, ratio=0.0000)
│   │   │       ├── 4: (action=1, visits=1, reward=0, ratio=0.0000)
│   │   │       ├── 1: (action=2, visits=1, reward=0, ratio=0.0000)
│   │   │       ├── 0: (action=3, visits=1, reward=0, ratio=0.0000)
│   │   │       └── 0: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   ├── 4: (action=1, visits=15, reward=0, ratio=0.0000)
│   │   │   ├── 5: (action=2, visi

error: display Surface quit

In [12]:
import random
from gym.envs.registration import register
import gym
from math import sqrt
import json

def init_env():
    register(
        id='FrozenLakeNotSlippery-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name': '4x4', 'is_slippery': False}
    )
    return gym.make('FrozenLakeNotSlippery-v0')

def main():
    random.seed(2)
    env = init_env()
    steps_values = [1000, 5000, 10000]  # Example steps values
   

    results = []

    for steps in steps_values:
        
        print(f"\nRunning MCTS with steps: {steps}\n")

        tree = Tree()
        monteCarloTreeSearch = MonteCarloTreeSearch(env=env, tree=tree)

        for _ in range(0, steps):
            env.reset()
            node = monteCarloTreeSearch.tree_policy()
            reward = monteCarloTreeSearch.default_policy(node)
            monteCarloTreeSearch.backward(node, reward)

        # Measure and display tree statistics
        tree_size = tree.size()
        max_depth, avg_depth = tree.depth_info()

        print(f"Tree size: {tree_size}")
        print(f"Maximum depth: {max_depth}")
        print(f"Average depth: {avg_depth:.2f}")

        monteCarloTreeSearch.tree.show()
        monteCarloTreeSearch.forward()
        policy_path = monteCarloTreeSearch.render_policy()

        print(f"Final policy path: {policy_path}")

        # Save the results for this combination
        results.append({
            'steps': steps,
            'tree_size': tree_size,
            'max_depth': max_depth,
            'avg_depth': avg_depth,
            'policy_path': policy_path
        })

    # Save results to a JSON file
    with open('simulations/mcts_results.json', 'w') as json_file:
        json.dump(results, json_file, indent=4)

if __name__ == "__main__":
    main()


c:\Users\cater\anaconda3\envs\cidl\Lib\site-packages\gym\envs\registration.py:498: UserWarning: WARN: Overriding environment FrozenLakeNotSlippery-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")



Running MCTS with steps: 1000

Tree size: 697
Maximum depth: 6
Average depth: 4.50
(0, {'prob': 1}): (action=None, visits=1000, reward=13, ratio=0.0130)
├── 0: (action=0, visits=244, reward=1, ratio=0.0041)
│   ├── 1: (action=2, visits=68, reward=1, ratio=0.0147)
│   │   ├── 1: (action=3, visits=18, reward=0, ratio=0.0000)
│   │   │   ├── 1: (action=3, visits=4, reward=0, ratio=0.0000)
│   │   │   │   ├── 0: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 5: (action=1, visits=1, reward=0, ratio=0.0000)
│   │   │   │   └── 1: (action=3, visits=1, reward=0, ratio=0.0000)
│   │   │   ├── 0: (action=0, visits=4, reward=0, ratio=0.0000)
│   │   │   │   ├── 4: (action=1, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 0: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   │   └── 1: (action=2, visits=1, reward=0, ratio=0.0000)
│   │   │   ├── 5: (action=1, visits=5, reward=0, ratio=0.0000)
│   │   │   └── 2: (action=2, visits=4, reward=0, ratio=0.0000)
│   │   │ 